<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 reddits by using a hashtag (you can use any hashtag) from Reddits.


In [1]:
# Write your code here


import requests
from bs4 import BeautifulSoup
import csv

# Define the URL of the IMDb page with user reviews
url = "https://www.imdb.com/title/tt6791350/reviews/?ref_=tt_ql_2"

# Create a CSV file to store the data
csv_filename = "imdb_reviews.csv"
csv_file = open(csv_filename, 'w', encoding='utf-8', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Title", "User", "Date", "Rating", "Review"])

# Function to scrape and save reviews
def scrape_reviews(url, csv_writer):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all("div", class_="lister-item-content")

        for review in reviews:
            title = review.find("a", class_="title").text.strip()
            user = review.find("span", class_="display-name-link").text.strip()
            date = review.find("span", class_="review-date").text.strip()
            rating = review.find("span", class_="rating-other-user-rating").text.strip()
            review_text = review.find("div", class_="text").text.strip().replace("\n", " ")
            csv_writer.writerow([title, user, date, rating, review_text])

# Scrape multiple pages of reviews (adjust the range accordingly)
for page_num in range(1, 500):
    page_url = f"{url}&start={page_num * 10}"
    scrape_reviews(page_url, csv_writer)

csv_file.close()
print("Reviews have been scraped and saved to", csv_filename)

Reviews have been scraped and saved to imdb_reviews.csv


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\soumya nanditha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [5]:
# Write your code here

import csv
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download NLTK data (if not already downloaded)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Initialize the stopwords, stemmer, and lemmatizer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define a function to clean the text
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text and remove stopwords
    words = [word for word in text.lower().split() if word not in stop_words]
    # Stem and lemmatize each word
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
    # Join the cleaned words back into a string
    return ' '.join(words)

# Load the CSV data into a pandas DataFrame
csv_filename = "imdb_reviews.csv"
df = pd.read_csv(csv_filename)

# Clean and preprocess the 'Review' column
df['Cleaned Review'] = df['Review'].apply(clean_text)

# Save the DataFrame with cleaned data to a new CSV file
cleaned_csv_filename = "imdb_reviews_cleaned.csv"
df.to_csv(cleaned_csv_filename, index=False, encoding='utf-8')

print("Text data has been cleaned and saved to", cleaned_csv_filename)



[nltk_data] Downloading package stopwords to C:\Users\soumya
[nltk_data]     nanditha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\soumya
[nltk_data]     nanditha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\soumya
[nltk_data]     nanditha\AppData\Roaming\nltk_data...


Text data has been cleaned and saved to imdb_reviews_cleaned.csv


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes:

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [8]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: typer<0.10.0,>=0.3.0 in c:\users\soumya nanditha\anaconda3\lib\site-packages (from spacy>=2.2.0->en-core-web-sm==2.2.0) (0.9.0)
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019121 sha256=a81936853806a3c4b885af43b50299dcbc632739d3431e683ebe3cc7db6df93a
  Stored in directory: c:\users\soumya nanditha\appdata\local\pip\cache\wheels\02\87\47\4d729a97cc46afa46135595b4de32d01461f05947df39166d7
Successfully built en-core-web-sm



In [15]:
pip install en_core_web_sm

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

  Using cached spacy-3.0.9-cp39-cp39-win_amd64.whl (11.1 MB)
  Using cached thinc-8.0.17-cp39-cp39-win_amd64.whl (1.0 MB)
  Created wheel for en-core-web-sm: filename=en_core_web_sm-3.0.0-py3-none-any.whl size=13704321 sha256=bbf411156c735bc56d38c9a360b27461c950331e6e2b8c630a185b32019b1c6e
  Stored in directory: c:\users\soumya nanditha\appdata\local\pip\cache\wheels\88\16\62\706709a80d48c983fffc44547114a933c464cd032e19378fb2
Successfully built en-core-web-sm
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.1
    Uninstalling thinc-8.2.1:
      Successfully uninstalled thinc-8.2.1



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\soumya nanditha\\anaconda3\\Lib\\site-packages\\~hinc\\backends\\cblas.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install spacy

In [7]:
# Write your code here

import spacy
from collections import Counter

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# Read the cleaned text from the CSV file
with open("imdb_reviews_cleaned.csv", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Initialize counters for parts of speech
noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

# Initialize counters for named entities
entity_counts = Counter()

# Process and analyze the text
for line in lines[1:]:  # Skip the header row
    parts = line.split(",")
    cleaned_review = parts[-1]  # Assuming the cleaned text is in the last column

    # Process the text with spaCy
    doc = nlp(cleaned_review)

    # Perform POS tagging and count different parts of speech
    for token in doc:
        if token.pos_ == "NOUN":
            noun_count += 1
        elif token.pos_ == "VERB":
            verb_count += 1
        elif token.pos_ == "ADJ":
            adj_count += 1
        elif token.pos_ == "ADV":
            adv_count += 1

    # Perform named entity recognition and count entities
    for ent in doc.ents:
        entity_counts[ent.label_] += 1

# Print the results
print("Parts of Speech Analysis:")
print("Nouns:", noun_count)
print("Verbs:", verb_count)
print("Adjectives:", adj_count)
print("Adverbs:", adv_count)

print("\nNamed Entity Recognition:")
for label, count in entity_counts.items():
    print(f"{label}: {count}")

# Dependency parsing and constituency parsing can be complex and require separate visualization tools
# We will not print the trees here, but you can use spaCy's visualization capabilities for these tasks




OSError: [E053] Could not read config file from C:\Users\soumya nanditha\anaconda3\lib\site-packages\en_core_web_sm\en_core_web_sm-2.2.0\config.cfg

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):**

Dependency parsing tree and Constituency parsing trees play a key role in semantics analysis, sentence structure analysis.Dependency parsing considers each word of sentence as a node and grammatical connections between different words as links.It follows a bottom-up approach.Whereas constituency parsing is based on context-free grammar and it follows a top-down approach.

Constituency parsing tree indicates how words in a sentence can be merged/grouped together to form different sentences.It follows a hierarchical model.It can be used in actively identifying and classifying POS in sentences and tagging them and helps in forming clusters of them.

Unlike Constituency parsing tree, dependency parsing tree gives more importance to connection between words and identifies various types of dependencies like subject-verb etc..This can prove useful in tasks like information extraction, sentiment analysis, ML etc..
These both are useful to bettter understand the structure of a sentence.